# Description

# Import needed modules

In [1]:
#you can remove stop words for speed
from nltk.corpus import stopwords 
# Helps to find the similarity between two vectors.
from nltk.cluster.util import cosine_distance 
import numpy as np
# used for the creation, manipulation, and study of complex networks or graphs.
import networkx as nx

# Open file and split into sentences

In [2]:
textfiles = [
    r"C:\Users\Ramachandraiah\Downloads\tamil (1).txt",
    r"C:\Users\Ramachandraiah\Downloads\kannada.txt",
    r"C:\Users\Ramachandraiah\Downloads\telugu.txt",
    r"C:\Users\Ramachandraiah\Downloads\spanish.txt",
    r"C:\Users\Ramachandraiah\Downloads\japanese.txt"
]

# Create lists to hold sentences for each file
sentences_lists = [[] for _ in range(len(textfiles))]

for index, filepath in enumerate(textfiles):
    with open(filepath, "r", encoding="utf-8") as file:
        # This file contains one paragraph of multiple sentences
        filedata = file.readline().strip()  # Read only the first line (paragraph)
        article = filedata.split(". ")  # Split paragraph into sentences
        for sentence in article:
            sentences_lists[index].append(sentence.replace("[^a-zA-Z]", " ").split(" "))

In [3]:
sentences_lists

[[['\ufeffஉலகில்',
   'மிக',
   'நீண்ட',
   'காலமாக',
   'வாழும்',
   'செம்மொழிகளில்',
   'தமிழ்',
   'ஒன்றாகும்.[10][11]',
   'A'],
  ['K'],
  ['ராமானுஜன்',
   'இதை',
   '"சமகால',
   'இந்தியாவின்',
   'ஒரே',
   'மொழி,',
   'இது',
   'ஒரு',
   'பாரம்பரிய',
   'கடந்த',
   'காலத்துடன்',
   'தொடர்ந்து',
   'அடையாளம்',
   'காணக்கூடியது"',
   'என்று',
   'விவரித்தார்.[12]',
   'பாரம்பரிய',
   'தமிழ்',
   'இலக்கியத்தின்',
   'பல்வேறு',
   'மற்றும்',
   'தரம்',
   '"உலகின்',
   'சிறந்த',
   'பாரம்பரிய',
   'மரபுகள்',
   'மற்றும்',
   'இலக்கியங்களில்',
   'ஒன்று"',
   'என்று',
   'விவரிக்கப்படுவதற்கு',
   'வழிவகுத்தது.[13]',
   'பதிவு',
   'செய்யப்பட்ட',
   'தமிழ்',
   'இலக்கியங்கள்',
   '2000',
   'ஆண்டுகளுக்கும்',
   'மேலாக',
   'ஆவணப்படுத்தப்பட்டுள்ளன.[14]',
   'தமிழ்',
   'இலக்கியத்தின்',
   'ஆரம்ப',
   'காலம்,',
   'சங்க',
   'இலக்கியம்,',
   'கி.பி'],
  ['கிமு',
   '300',
   'முதல்',
   'கிபி',
   '300',
   'வரை.[15][16]',
   'இது',
   'திராவிட',
   'மொழிகளில்',
   'மிகப்',
   'பழமையான',

In [4]:
for i in range(len(sentences_lists)):
    print(f'-------------Text{i}--------------')
    print(' '*10)
    print(sentences_lists[i])
    print(' '*10)

-------------Text0--------------
          
[['\ufeffஉலகில்', 'மிக', 'நீண்ட', 'காலமாக', 'வாழும்', 'செம்மொழிகளில்', 'தமிழ்', 'ஒன்றாகும்.[10][11]', 'A'], ['K'], ['ராமானுஜன்', 'இதை', '"சமகால', 'இந்தியாவின்', 'ஒரே', 'மொழி,', 'இது', 'ஒரு', 'பாரம்பரிய', 'கடந்த', 'காலத்துடன்', 'தொடர்ந்து', 'அடையாளம்', 'காணக்கூடியது"', 'என்று', 'விவரித்தார்.[12]', 'பாரம்பரிய', 'தமிழ்', 'இலக்கியத்தின்', 'பல்வேறு', 'மற்றும்', 'தரம்', '"உலகின்', 'சிறந்த', 'பாரம்பரிய', 'மரபுகள்', 'மற்றும்', 'இலக்கியங்களில்', 'ஒன்று"', 'என்று', 'விவரிக்கப்படுவதற்கு', 'வழிவகுத்தது.[13]', 'பதிவு', 'செய்யப்பட்ட', 'தமிழ்', 'இலக்கியங்கள்', '2000', 'ஆண்டுகளுக்கும்', 'மேலாக', 'ஆவணப்படுத்தப்பட்டுள்ளன.[14]', 'தமிழ்', 'இலக்கியத்தின்', 'ஆரம்ப', 'காலம்,', 'சங்க', 'இலக்கியம்,', 'கி.பி'], ['கிமு', '300', 'முதல்', 'கிபி', '300', 'வரை.[15][16]', 'இது', 'திராவிட', 'மொழிகளில்', 'மிகப்', 'பழமையான', 'இலக்கியங்களைக்', 'கொண்டுள்ளது'], ['பாறை', 'ஆணைகள்', 'மற்றும்', "'ஹீரோ", "ஸ்டோன்கள்'", 'ஆகியவற்றில்', 'காணப்படும்', 'ஆரம்பகால', 'கல்வெட்டுப்', 'பதிவுகள்',

In [5]:
# step-2 to find similarity between two sentences 
def sentence_similarity(sent1, sent2 ):
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    # build the vector for the first sentence
    for w in sent1:
        vector1[all_words.index(w)] += 1
        # build the vector for the second sentence
    for w in sent2:
        vector2[all_words.index(w)] += 1
    return 1 - cosine_distance(vector1, vector2)

# Create the similarity matrix 

In [6]:
import numpy as np
similarity=[]
for i in sentences_lists:
    similarity_matrix = np.zeros((len(i), len(i)))
    for idx1 in range(len(i)):
        for idx2 in range(len(i)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(i[idx1], i[idx2])
    similarity.append(similarity_matrix)

In [7]:
similarity

[array([[0.        , 0.        , 0.12403473, 0.        , 0.        ,
         0.04761905],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.12403473, 0.        , 0.        , 0.03202563, 0.04606534,
         0.15947323],
        [0.        , 0.        , 0.03202563, 0.        , 0.04794633,
         0.03688556],
        [0.        , 0.        , 0.04606534, 0.04794633, 0.        ,
         0.13263953],
        [0.04761905, 0.        , 0.15947323, 0.03688556, 0.13263953,
         0.        ]]),
 array([[0.        , 0.17624205],
        [0.17624205, 0.        ]]),
 array([[0.        , 0.20412415, 0.0727393 ],
        [0.20412415, 0.        , 0.2227177 ],
        [0.0727393 , 0.2227177 , 0.        ]]),
 array([[0.        , 0.40857249, 0.20519567],
        [0.40857249, 0.        , 0.38851434],
        [0.20519567, 0.38851434, 0.        ]]),
 array([[0.]])]

# Get the pagerank scores

In [8]:
# Step 3 - Rank sentences in similarity martix
score=[]
for i in similarity:
    sentence_similarity_graph = nx.from_numpy_array(i)
    scores = nx.pagerank(sentence_similarity_graph)
    score.append(scores)

In [9]:
score

[{0: 0.13784810907858028,
  1: 0.02912621359223663,
  2: 0.2697692147249517,
  3: 0.10484779714633141,
  4: 0.17865139728330745,
  5: 0.2797572681745926},
 {0: 0.5, 1: 0.5},
 {0: 0.28266389350659626, 1: 0.41857092197752055, 2: 0.29876518451588335},
 {0: 0.3086543818872344, 1: 0.3914534356013062, 2: 0.29989218251145927},
 {0: 1.0}]

# Sort sentences by pagerank

In [10]:
ranked_sentences = []
for sentence_list in sentences_lists:
    ranked_sentence = sorted(((score, sentence) for score, sentence in zip(scores, sentence_list)), reverse=True)
    ranked_sentences.append(ranked_sentence)

In [11]:
ranked_sentences

[[(0,
   ['\ufeffஉலகில்',
    'மிக',
    'நீண்ட',
    'காலமாக',
    'வாழும்',
    'செம்மொழிகளில்',
    'தமிழ்',
    'ஒன்றாகும்.[10][11]',
    'A'])],
 [(0,
   ['\ufeffಕನ್ನಡವು',
    'ದಕ್ಷಿಣ,',
    'ಮಧ್ಯ',
    'ಭಾರತ',
    'ಮತ್ತು',
    'ಡೆಕ್ಕನ್',
    'ಪ್ರಸ್ಥಭೂಮಿಯ',
    'ಹಲವಾರು',
    'ರಾಜವಂಶಗಳು',
    'ಮತ್ತು',
    'ಸಾಮ್ರಾಜ್ಯಗಳ',
    'ಆಸ್ಥಾನ',
    'ಭಾಷೆಯಾಗಿತ್ತು,',
    'ಅವುಗಳೆಂದರೆ',
    'ಕದಂಬ',
    'ರಾಜವಂಶ,',
    'ಪಶ್ಚಿಮ',
    'ಗಂಗ',
    'ರಾಜವಂಶ,',
    'ಚಾಲುಕ್ಯ',
    'ರಾಜವಂಶ,',
    'ರಾಷ್ಟ್ರಕೂಟ',
    'ಸಾಮ್ರಾಜ್ಯ,',
    'ಪಶ್ಚಿಮ',
    'ಚಾಲುಕ್ಯ',
    'ಸಾಮ್ರಾಜ್ಯ',
    'ಯಾದವ',
    'ರಾಜವಂಶ',
    'ಅಥವಾ',
    'ಸೇನರು,',
    'ಮೈಸೂರು',
    'ಸಾಮ್ರಾಜ್ಯ,',
    'ನಾಯಕರ',
    'ಕೆಳದಿ,[10]',
    'ಹೊಯ್ಸಳ',
    'ರಾಜವಂಶ',
    'ಮತ್ತು',
    'ವಿಜಯನಗರ',
    'ಸಾಮ್ರಾಜ್ಯ'])],
 [(0,
   ['\ufeffలుగు',
    '(/ˈtɛlʊɡuː/;[6]',
    'తెలుగు,',
    'తెలుగు',
    'ఉచ్చారణ:',
    '[ˈt̪eluɡu])',
    'అనేది',
    'భారతదేశంలోని',
    'ఆంధ్ర',
    'ప్రదేశ్',
    'మరియు',
    'తెలంగాణా',
    'రాష్ట్రాలకు',
    'చెందిన',
    'ద్రావిడ',
  

In [ ]:
# Finish off by printing summary.

for i in ranked_sentences:
    n = int(input("How many sentences do you want in the summary? "))
  
    for i in range(n):
        print("".join(ranked_sentence[i][1]))

How many sentences do you want in the summary? 0
How many sentences do you want in the summary? 1
﻿日本語(日本語、Nihongo、[ɲihoŋɡo]ⓘ)は、日本人が話すジャポニック語族の主要言語です。言語話者は約1億2,000万人で、主に言語が国語である唯一の国である日本と、世界中に離散した日本人がいます。
